In [ ]:
import pandas as pd
import json
import numpy as np

# make data

In [ ]:
with open('data/zac2022_train_merged_final.json',encoding='utf-8') as f:
    data=json.load(f)
    df = pd.json_normalize(data,'data')

In [ ]:
df.head()

,id,question,title,text,category,is_long_answer,short_candidate_start,short_candidate,answer
0,718d41cd997b2b44b0685ac54aa55bd8,Thủ tướng Trung Quốc là gì,Trung Quốc,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...,PARTIAL_ANNOTATION,True,NaN,NaN,NaN
1,c926e7b0717202618a10dd907d4b4c39,Đất nước nào không có quân đội,,"có 23 quốc gia không có lực lượng quân đội, ba...",FULL_ANNOTATION,True,53.0,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",wiki/Danh_sách_quốc_gia_không_có_lực_lượng_vũ_...
2,d38ef5bf1fb82b410026ed82c8a44cae,Pháp tấn công xâm lược Việt Nam vào ngày tháng...,Raymondienne,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,FALSE_LONG_ANSWER,False,NaN,NaN,NaN
3,b6b5589a98fdccd208dc752bac853993,Cờ vua còn có tên gọi nào khác,Cúp cờ vua thế giới,Cúp cờ vua thế giới là tên gọi một số giải đấu...,FALSE_LONG_ANSWER,False,NaN,NaN,NaN
4,82396a18fa9812bfec4d3ecb7ae60905,Núi nào cao nhất châu âu,Shkhara,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,FULL_ANNOTATION,True,73.0,núi Elbrus,wiki/Elbrus


In [ ]:
import re

def checkdate(date):
    # reg1 = ngày .. tháng .. năm ..
    # reg2 = tháng .. năm ..
    # reg3 = năm ..
    # reg4 = ngày .. tháng ..
    # check date by reg1, reg2, reg3, reg4
    reg1 = r'ngày\s\d{1,2}\stháng\s\d{1,2}\snăm\s\d{4}'
    reg2 = r'tháng\s\d{1,2}\snăm\s\d{4}'
    reg3 = r'năm\s\d{4}'
    reg4 = r'ngày\s\d{1,2}\stháng\s\d{1,2}'
    if re.match(reg1, date) or re.match(reg2, date) or re.match(reg3, date) or re.match(reg4, date):
        return True
    return False

def appy_func(x):
    if x.startswith('wiki/'):
        return 'wiki'
    if x.isdigit():
        return 'number'
    return 'text'

full_df = df.copy()
# full_df = df[df['category']=='FULL_ANNOTATION']

# loop through all row
# create new empty column
full_df['train_label'] = ''


date_keyword = [
    'ngày nào',
    'tháng nào',
    'năm nào',
    'thế nào',
    'khi nào',
    'lúc nào'
]
for index, row in full_df.iterrows():
    for date in date_keyword:
        if date in row['question'].lower():
            full_df.at[index, 'train_label'] = 'text'
            break

    if 'bao nhiêu' in row['question'].lower() and 'năm bao nhiêu' not in row['question'].lower():
        full_df.loc[index, 'train_label'] = 'number'

    if row['category']=='FULL_ANNOTATION':
        full_df.loc[index, 'train_label'] = appy_func(row['answer'])
full_df = full_df[full_df['train_label']!='']
# full_df['train_label'] = full_df['answer'].apply(appy_func)

In [ ]:
print('bao nhiêu', len(df[df['question'].str.lower().str.contains('bao nhiêu')]))

bao nhiêu 2228


In [ ]:
for keyword in date_keyword:
    print(keyword, len(df[df['question'].str.lower().str.contains(keyword)]))

ngày nào 158
tháng nào 7
năm nào 1463
thế nào 57
khi nào 137
lúc nào 3


In [ ]:
full_df['train_label'].value_counts()

wiki      4608
number    2010
text      1885
Name: train_label, dtype: int64

In [ ]:
train_df = full_df[['question','train_label']]
# question
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s
# train_df['question'] = train_df['question'].apply(remove_accents)

# label
LABEL={
	'wiki':0,
	'number':1,
	'text':2
}
train_df['train_label'] = train_df['train_label'].apply(lambda x: LABEL[x])
train_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,question,train_label
1,Đất nước nào không có quân đội,0
2,Pháp tấn công xâm lược Việt Nam vào ngày tháng...,2
4,Núi nào cao nhất châu âu,0
8,Hà Nội đã trải qua bao nhiêu lần đổi tên,1
9,Thành phố nào là thủ phủ của Ai Cập trong đế q...,0


In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [ ]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)


In [ ]:
def dataset_df(df,test_size=0.2):
  # download & load 20newsgroups dataset from sklearn's repos
  dataset = df
  documents = dataset.question
  labels = dataset.train_label
  # split into training & testing a return data as well as label names
  return train_test_split(documents.tolist(), labels.tolist(), test_size=test_size), np.unique(dataset.train_label)

In [ ]:
(train_texts, valid_texts, train_labels, valid_labels), target_names = dataset_df(train_df)

In [ ]:
train_texts

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)


In [ ]:
import torch

In [ ]:
class ZaloDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = ZaloDataset(train_encodings, train_labels)
valid_dataset = ZaloDataset(valid_encodings, valid_labels)

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer

In [ ]:
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6802
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2553
  Number of trainable parameters = 109484547


Step,Training Loss,Validation Loss,Accuracy
400,0.412000,0.174438,0.968254
800,0.109600,0.088795,0.986479
1200,0.092300,0.107291,0.981775
1600,0.075900,0.079256,0.987654
2000,0.058700,0.077786,0.987654
2400,0.046900,0.076147,0.988242


***** Running Evaluation *****
  Num examples = 1701
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1701
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1701
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1701
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1600
Configuration saved in ./results/checkpoint-1600/config.json
Model weights saved in ./results/checkpoint-1600/pytorch_model.bi

TrainOutput(global_step=2553, training_loss=0.12687685384126537, metrics={'train_runtime': 426.4598, 'train_samples_per_second': 47.85, 'train_steps_per_second': 5.986, 'total_flos': 681623058867660.0, 'train_loss': 0.12687685384126537, 'epoch': 3.0})

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

In [ ]:
for i in question:
  print(i,get_prediction(i))

In [ ]:
get_prediction("Có mấy phím trên bàn phím piano")

1

In [ ]:
trainer.save_model("model")

Saving model checkpoint to model
Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin


In [ ]:
!zip -r "model1.zip" "model" 

  adding: model/ (stored 0%)
  adding: model/pytorch_model.bin (deflated 7%)
  adding: model/training_args.bin (deflated 48%)
  adding: model/config.json (deflated 51%)
